In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import random

In [ ]:
path = '../Data/Images'
folders = os.listdir(path)

In [ ]:
data = []

In [3]:
data = get_data()

In [ ]:
for folder in folders:
    
    # Label as healthy or diseased
    if folder.endswith('healthy'):
        label = 1
    else:
        label = 0
    
    # Iterate through the folders
    folder_images = os.listdir(path + '/' + folder)
    folder_path = path + '/' + folder + '/'
    
    # Extract images from each folder
    for image in folder_images:
        image_path = os.path.join(folder_path, image)
        image_path = Image.open(image_path)
        
        # Convert image to pixel values
        pixel_values = np.array(image_path)
        
        # Add image and label
        data.append((pixel_values, label))

In [ ]:
len(data)

In [ ]:
data = data[:10000]

In [ ]:
# Shuffle the data
random.shuffle(data)

In [ ]:
images = [pair[0] for pair in data]
labels = [pair[1] for pair in data]

In [ ]:
# Define the proportions for train, validation, and test sets
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

In [ ]:
# Calculate the number of samples for each set
num_samples = len(data)
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

In [ ]:
# Split the data
train_images = images[:num_train]
val_images = images[num_train:num_train + num_val]
test_images = images[num_train + num_val:]

# Split the data
train_labels = labels[:num_train]
val_labels = labels[num_train:num_train + num_val]
test_labels = labels[num_train + num_val:]

In [ ]:
len(train_labels)

In [ ]:
def image_mean_and_var(images):
    
    mean = np.mean(images, axis=0)
    variance = np.var(images, axis=0)
    
    return mean, variance

In [ ]:
mean, var = normalise_images(train_images)

In [ ]:
def apply_normalisation(images, mean, variance):
    
    images = (images - mean) / np.sqrt(variance)
    
    return images

In [ ]:
#train_images = apply_normalisation(train_images, mean, var)

In [ ]:
type(train_images)

In [ ]:
# Normalize the training data
train_images = [image/255 for image in train_images]
val_images = [image/255 for image in val_images]
test_images = [image/255 for image in test_images]

In [ ]:
# check val and test mean and variance 

In [ ]:
img = train_images[2]

In [ ]:
plt.imshow(img);
plt.axis('off');  # Hide axis
plt.show();

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
t = train_images[:1000]
l = train_labels[:1000]

In [ ]:
train_data = [(torch.tensor(t[i]), torch.tensor(l[i])) for i in range(1000)]

In [ ]:
#Define train and test loaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, drop_last=True)

In [ ]:
for batch_tensor1, batch_tensor2 in train_loader:
    # Print the size of the current batch tensors
    print("Batch tensor 1 size:", batch_tensor1.size())
    print("Batch tensor 2 size:", batch_tensor2.size())

In [ ]:
# Define the CNN model using nn.Sequential
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Conv2d(32, 64, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Flatten(),
    nn.Linear(3136, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

In [ ]:
# Instantiate the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [ ]:
def accuracy(model, images, test_loader):

    # Evaluation
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    return accuracy

In [ ]:
# Training lop
num_epochs = 1
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

    acc = accuracy(model, images, test_loader)

    print(f'Test Accuracy: {acc * 100:.2f}%')